In [ ]:
# Simple Stacking Ensemble Model
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, cross_val_predict
from sklearn.preprocessing import LabelEncoder
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
import xgboost as xgb
import catboost as cb
import lightgbm as lgb

# Load data
df = pd.read_csv('human_vital_signs_dataset_2024.csv')

# Prepare data
features = ['Heart Rate', 'Body Temperature', 'Oxygen Saturation',
            'Systolic Blood Pressure', 'Diastolic Blood Pressure',
            'Age', 'Gender', 'Weight (kg)', 'Height (m)', 'Derived_BMI']
target = 'Risk Category'

df['Gender'] = df['Gender'].map({'Male': 1, 'Female': 0})
df['Risk'] = df[target].map({'Low Risk': 0, 'High Risk': 1})

X = df[features]
y = df['Risk']

# Split data
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

print("=" * 50)
print("STACKING ENSEMBLE MODEL")
print("=" * 50)
print(f"Training samples: {len(X_train)}")
print(f"Test samples: {len(X_test)}")

# Step 1: Train base models
print("\nTraining base models...")

# XGBoost
xgb_model = xgb.XGBClassifier(n_estimators=100, random_state=42,
                              use_label_encoder=False, eval_metric='logloss')
xgb_model.fit(X_train, y_train)
xgb_preds = xgb_model.predict_proba(X_test)[:, 1]

# CatBoost
cat_model = cb.CatBoostClassifier(iterations=100, random_seed=42, verbose=0)
cat_model.fit(X_train, y_train, cat_features=['Gender'])
cat_preds = cat_model.predict_proba(X_test)[:, 1]

# LightGBM
lgb_model = lgb.LGBMClassifier(n_estimators=100, random_state=42)
lgb_model.fit(X_train, y_train)
lgb_preds = lgb_model.predict_proba(X_test)[:, 1]

# Step 2: Create meta-features
meta_X = np.column_stack([xgb_preds, cat_preds, lgb_preds])

# Step 3: Train meta-model (Logistic Regression)
meta_model = LogisticRegression(random_state=42)
meta_model.fit(meta_X, y_test)

# Step 4: Make final predictions
final_preds = meta_model.predict(meta_X)

# Evaluate
print("\nStacking Model Results:")
print("-" * 40)
print(f"Accuracy:  {accuracy_score(y_test, final_preds):.4f}")
print(f"Precision: {precision_score(y_test, final_preds):.4f}")
print(f"Recall:    {recall_score(y_test, final_preds):.4f}")
print(f"F1-Score:  {f1_score(y_test, final_preds):.4f}")

# Compare with individual models
print("\n" + "=" * 50)
print("INDIVIDUAL MODEL COMPARISON")
print("=" * 50)

models = {
    'XGBoost': xgb_model,
    'CatBoost': cat_model,
    'LightGBM': lgb_model,
    'Stacking': 'N/A'
}

for name, model in models.items():
    if name != 'Stacking':
        preds = model.predict(X_test)
        acc = accuracy_score(y_test, preds)
        print(f"{name:10s}: Accuracy = {acc:.4f}")

print(f"Stacking   : Accuracy = {accuracy_score(y_test, final_preds):.4f}")


STACKING ENSEMBLE MODEL
Training samples: 160016
Test samples: 40004

Training base models...
[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Info] Number of positive: 84166, number of negative: 75850
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.019143 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 1443
[LightGBM] [Info] Number of data points in the train set: 160016, number of used features: 10
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.525985 -> initscore=0.104033
[LightGBM] [Info] Start training from score 0.104033

Stacking Model Results:
----------------------------------------
Accuracy:  0.9995
Precision: 0.9995
Recall:    0.9996
F1-Score:  0.9995

INDIVIDUAL MODEL COMPARISON
XGBoost   : Accuracy = 0.9987
CatBoost  : Accuracy = 0.9987
LightGBM  : Accuracy = 0.9985
Stacking   : Accuracy = 0.9995
